# Collect Data (Game 3)

In [2]:
import requests
import json
import pandas as pd
import numpy as np
from geopy import geocoders
from geopy.geocoders import Nominatim
import plotly.graph_objects as go
import plotly.express as px
import chart_studio.plotly as py
from plotly.offline import iplot
import os
from pandas.io.json import json_normalize
# Importing re package for using regular expressions 
import re 

# NHL API:https://github.com/dword4/nhlapi
# https://www.dataquest.io/blog/python-api-tutorial/
# https://stackoverflow.com/questions/41100303/convert-api-to-pandas-dataframe

    
r = requests.get('https://statsapi.web.nhl.com/api/v1/game/2018030411/feed/live')


def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

def geo(location):
    g = geocoders.Nominatim(user_agent='MSGT_Lab5', timeout=None)
    try:
        loc = g.geocode(location)
        return loc.latitude, loc.longitude
    except:
        return 0,0

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html
# pandas data frame of allPlays
x = r.json()

# Allplays DataFrame
df = pd.DataFrame(x['liveData']['plays']['allPlays'])
df

# Creating variable for normalizing nested data
data = x['liveData']['plays']['allPlays']
data

# Flatten allPlays nested data
results_df = pd.json_normalize(data)
results_df

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html
# Coordinate single x,y
# df.iloc[5][2]

# Print coordinate Column w/out header
coords = df.iloc[:,2]

events = df.iloc[:,0][3:]

# Flatten Coordinates data
coords_df = pd.json_normalize(coords)
coords_df

# Flatten allPlays nested data
event_df = pd.json_normalize(events)

# event_df.description


period = df.iloc[:,1][2:]

period_df = pd.json_normalize(period)
period_df['period']



# team = df.iloc[:,4][2:]


time = df.iloc[:,1][2:]
time

time_df = pd.json_normalize(time)
time_df['periodTime']
time_df['dateTime']


date = time_df['dateTime'][:9].str[:10]
date

gamedata_df = pd.DataFrame(x['gameData'])
game_num = gamedata_df['game'][0]

# # df.loc[['Dean', 'Aaron'], 'food']
# gamedata_df.loc[['home', 'away'],'teams']

# away = []
# home = []

# away = gamedata_df.iloc[10,[3]]
# away_df = pd.json_normalize(away)
# away_df

home_away = gamedata_df.iloc[[10,11]]
home_away

home_away_df = pd.json_normalize(home_away.teams.to_dict())


home_away_df['away.name']
home_away_df['home.name']
# home_away_df['away.shortName']
# home_away_df['home.shortName']
# home_away_df['away.triCode']
# home_away_df['home.triCode']


# # df.loc[:, ['food', 'color']]
# gamedata_df.loc[:,['home', 'away']]

#create empty array for putting event coordinate data in
latitude = []
longitude = []

#iterative loop for pushing data into arrays
for i in coords:

    if coords_df['x'] is not None:

        lat = coords_df['y']
        lon = coords_df['x']
        for i in [lat]:
            latitude.append(i)
        for i in [lon]:
            longitude.append(i)

# Creates datafrome with events that includes coordinates
df = pd.DataFrame(list(zip(results_df['result.event'],results_df['result.description'],results_df['team.name'],results_df['team.triCode'],period_df['period'],time_df['periodTime'],time_df['dateTime'].str[:10],lat,lon)), columns = ['Event','Description','Team','Tricode','Period','Time','Date','Lat','Lon'])
# Removes rows with "NaN" stoppages and period ends (https://stackoverflow.com/questions/39339935/pandas-dropping-rows-with-missing-data-not-working-using-isnull-notnull)
df.replace(["NaN", 'NaT'], np.nan, inplace = True)
df = df.dropna()

# must correct latitude to 95 or -95!
#     https://datatofish.com/compare-values-dataframes/

# Create a column comparing tricode of Dataframe to tricode of API and tell me which team is home/away
df['Home_Away_Team'] = np.where(df['Tricode'].values == home_away_df['home.triCode'].values, 'Away', 'Home')

# Create a column comparing tricode of Dataframe to tricode of API and assign 2 to Away team and 0 to Home team
# If I assign 1 it will assign "True" to first period events in df['Swap']
df['AssigningSides'] = np.where(df['Tricode'].values == home_away_df['home.triCode'].values, 2, 0)

# Create a column that returns True when Period matches assigned '2'
df['Swap'] = np.where(df['Period'].values == df['AssigningSides'].astype(int), 'True', 'False')


# Create new column for Lon (home switch) in 2nd Period
# Used the following URL to make LON opposite (df['Lon'].abs() - doesn't work because only accounts for removing the negative)
# https://stackoverflow.com/questions/25315549/make-every-numeric-value-in-a-pandas-dataframe-negative


# Create new column that returns opposite longitude for everything during 2nd Period (*-1)
# df['Lon_2ndPer'] = np.where(df['Period'] == 2, df['Lon']*-1, df['Lon'])
#Transpose the New column over the old "Lon"
df['Lon'] = np.where(df['Period'] == 2, df['Lon']*-1, df['Lon'])

# #Transpose the new column over the old "lon" only during the 2nd period
# df['Lon'] = np.where(df['Period'] == 2, df['AwayLon_2ndPer'], )



# df['Lon'] = np.where(df['Period'] != 2, df['AwayLon_2ndPer'], 0.0)



# #NEW CODE (Makes everything on X-Axis Opposite)
# df['2ndPer_ZoneChange'] = np.where(df['Lon'] != 0, df['Lon']*-1, 0)


# #Transpose the New column over the old "Lon"
# df['Lon'] = np.where(df['Period'] == 2, df['Lon']*-1, 0)

pd.set_option("display.max_rows", 30)

df

# # SAVING TO FILE!
# df.to_csv('2019SCF_events.csv', header = True, mode = 'a')

,Event,Description,Team,Tricode,Period,Time,Date,Lat,Lon,Home_Away_Team,AssigningSides,Swap
3,Faceoff,Oskar Sundqvist faceoff won against Sean Kuraly,St. Louis Blues,STL,1,00:10,2019-05-28,0.0,0.0,Home,0,False
5,Faceoff,Sean Kuraly faceoff won against Oskar Sundqvist,Boston Bruins,BOS,1,00:28,2019-05-28,22.0,-20.0,Away,2,False
6,Shot,Charlie McAvoy Wrist Shot saved by Jordan Binn...,Boston Bruins,BOS,1,00:28,2019-05-28,-19.0,47.0,Away,2,False
8,Faceoff,Patrice Bergeron faceoff won against Brayden S...,Boston Bruins,BOS,1,00:35,2019-05-28,-22.0,69.0,Away,2,False
9,Shot,David Pastrnak Slap Shot saved by Jordan Binni...,Boston Bruins,BOS,1,00:35,2019-05-28,-5.0,47.0,Away,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...
314,Shot,Ryan O'Reilly Wrist Shot saved by Tuukka Rask,St. Louis Blues,STL,3,19:46,2019-05-28,29.0,-63.0,Home,0,False
316,Faceoff,Ryan O'Reilly faceoff won against Sean Kuraly,St. Louis Blues,STL,3,19:58,2019-05-28,22.0,-69.0,Home,0,False
317,Shot,Brayden Schenn Backhand saved by Tuukka Rask,St. Louis Blues,STL,3,19:59,2019-05-28,3.0,-81.0,Home,0,False
318,Blocked Shot,Joakim Nordstrom,Boston Bruins,BOS,3,20:00,2019-05-28,11.0,-69.0,Away,2,False


# Display Data

In [3]:
import requests
import json
import pandas as pd
import numpy as np
from geopy import geocoders
from geopy.geocoders import Nominatim
import plotly.graph_objects as go
import plotly.express as px
import chart_studio.plotly as py
from plotly.offline import iplot
import os
from pandas.io.json import json_normalize
# Importing re package for using regular expressions 
import re 

# NHL API:https://github.com/dword4/nhlapi
# https://www.dataquest.io/blog/python-api-tutorial/
# https://stackoverflow.com/questions/41100303/convert-api-to-pandas-dataframe

    
r = requests.get('https://statsapi.web.nhl.com/api/v1/game/2018030412/feed/live')


def jprint(obj):
    # create a formatted string of the Python JSON object
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

def geo(location):
    g = geocoders.Nominatim(user_agent='MSGT_Lab5', timeout=None)
    try:
        loc = g.geocode(location)
        return loc.latitude, loc.longitude
    except:
        return 0,0

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html
# pandas data frame of allPlays
x = r.json()

# Allplays DataFrame
df = pd.DataFrame(x['liveData']['plays']['allPlays'])
df

# Creating variable for normalizing nested data
data = x['liveData']['plays']['allPlays']
data

# Flatten allPlays nested data
results_df = pd.json_normalize(data)
results_df

# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iloc.html
# Coordinate single x,y
# df.iloc[5][2]

# Print coordinate Column w/out header
coords = df.iloc[:,2]

events = df.iloc[:,0][3:]

# Flatten Coordinates data
coords_df = pd.json_normalize(coords)
coords_df

# Flatten allPlays nested data
event_df = pd.json_normalize(events)

# event_df.description


period = df.iloc[:,1][2:]

period_df = pd.json_normalize(period)
period_df['period']



# team = df.iloc[:,4][2:]


time = df.iloc[:,1][2:]
time

time_df = pd.json_normalize(time)
time_df['periodTime']
time_df['dateTime']


date = time_df['dateTime'][:9].str[:10]
date

gamedata_df = pd.DataFrame(x['gameData'])
game_num = gamedata_df['game'][0]

# # df.loc[['Dean', 'Aaron'], 'food']
# gamedata_df.loc[['home', 'away'],'teams']

# away = []
# home = []

# away = gamedata_df.iloc[10,[3]]
# away_df = pd.json_normalize(away)
# away_df

home_away = gamedata_df.iloc[[10,11]]
home_away

home_away_df = pd.json_normalize(home_away.teams.to_dict())


home_away_df['away.name']
home_away_df['home.name']
# home_away_df['away.shortName']
# home_away_df['home.shortName']
# home_away_df['away.triCode']
# home_away_df['home.triCode']


# # df.loc[:, ['food', 'color']]
# gamedata_df.loc[:,['home', 'away']]

#create empty array for putting event coordinate data in
latitude = []
longitude = []

#iterative loop for pushing data into arrays
for i in coords:

    if coords_df['x'] is not None:

        lat = coords_df['y']
        lon = coords_df['x']
        for i in [lat]:
            latitude.append(i)
        for i in [lon]:
            longitude.append(i)

# Creates datafrome with events that includes coordinates
df = pd.DataFrame(list(zip(results_df['result.event'],results_df['result.description'],results_df['team.name'],results_df['team.triCode'],period_df['period'],time_df['periodTime'],time_df['dateTime'].str[:10],lat,lon)), columns = ['Event','Description','Team','Tricode','Period','Time','Date','Lat','Lon'])
# Removes rows with "NaN" stoppages and period ends (https://stackoverflow.com/questions/39339935/pandas-dropping-rows-with-missing-data-not-working-using-isnull-notnull)
df.replace(["NaN", 'NaT'], np.nan, inplace = True)
df = df.dropna()

# must correct latitude to 95 or -95!
#     https://datatofish.com/compare-values-dataframes/

# Create a column comparing tricode of Dataframe to tricode of API and tell me which team is home/away
df['Home_Away_Team'] = np.where(df['Tricode'].values == home_away_df['home.triCode'].values, 'Away', 'Home')

# Create a column comparing tricode of Dataframe to tricode of API and assign 2 to Away team and 0 to Home team
# If I assign 1 it will assign "True" to first period events in df['Swap']
df['AssigningSides'] = np.where(df['Tricode'].values == home_away_df['home.triCode'].values, 2, 0)

# Create a column that returns True when Period matches assigned '2'
df['Swap'] = np.where(df['Period'].values == df['AssigningSides'].astype(int), 'True', 'False')


# Create new column for Lon (home switch) in 2nd Period
# Used the following URL to make LON opposite (df['Lon'].abs() - doesn't work because only accounts for removing the negative)
# https://stackoverflow.com/questions/25315549/make-every-numeric-value-in-a-pandas-dataframe-negative

# This is only switching the Away Team.
# I could switch both Teams using the *-1 equation...?



# Create new column that returns opposite longitude for everything during 2nd Period (*-1)
# df['Lon_2ndPer'] = np.where(df['Period'] == 2, df['Lon']*-1, df['Lon'])
#Transpose the New column over the old "Lon"
df['Lon'] = np.where(df['Period'] == 2, df['Lon']*-1, df['Lon'])

df




# # SAVING TO FILE!
# coordinates_df.to_csv('2019SCF_events.csv', header = True, mode = 'a')
# #     coordinates_df.to_json(r'events.json')

# mapbox_access_token = os.getenv('')
mapbox_access_token = open("mapbox_token.txt").read() # Create a local file with mapbox token

# df = pd.read_csv('2019SCF_events.csv')


# All Events
events_lat = df.Lat
events_lon = df.Lon
locations_name = df.Description


# # Team Events
# STL = df[df['Team'].str.contains('St. Louis Blues')]
# STL_lat = df[df['Team'].str.contains('St. Louis Blues')]['Lat']
# STL_lon = df[df['Team'].str.contains('St. Louis Blues')]['Lon']
# STL_text = df[df['Team'].str.contains('St. Louis Blues')]['Description']

# BOS = df[df['Team'].str.contains('Boston Bruins')]
# BOS_lat = df[df['Team'].str.contains('Boston Bruins')]['Lat']
# BOS_lon = df[df['Team'].str.contains('Boston Bruins')]['Lon']
# BOS_text = df[df['Team'].str.contains('Boston Bruins')]['Description']

# Home Team Events
home = df[df['Home_Away_Team'].str.contains('Home')]
home_lat = df[df['Home_Away_Team'].str.contains('Home')]['Lat']
home_lon = df[df['Home_Away_Team'].str.contains('Home')]['Lon']
home_text = df[df['Home_Away_Team'].str.contains('Home')]['Description']


# Away Team Events
away = df[df['Home_Away_Team'].str.contains('Away')]
away_lat = df[df['Home_Away_Team'].str.contains('Away')]['Lat']
away_lon = df[df['Home_Away_Team'].str.contains('Away')]['Lon']
away_text = df[df['Home_Away_Team'].str.contains('Away')]['Description']

# Display Data for Goals
goal = df[df['Event'].str.contains('Goal')]
goal_lat = df[df['Event'].str.contains('Goal')]['Lat']
goal_lon = df[df['Event'].str.contains('Goal')]['Lon']
goal_text = df[df['Event'].str.contains('Goal')]['Description']

# Display Data for Hits
hits = df[df['Event'].str.contains('Hit')]
hits_lat = df[df['Event'].str.contains('Hit')]['Lat']
hits_lon = df[df['Event'].str.contains('Hit')]['Lon']
hits_text = df[df['Event'].str.contains('Hit')]['Description']


# Display Data for Faceoffs
faceoff = df[df['Event'].str.contains('Faceoff')]
faceoff_lat = df[df['Event'].str.contains('Faceoff')]['Lat']
faceoff_lon = df[df['Event'].str.contains('Faceoff')]['Lon']
faceoff_text = df[df['Event'].str.contains('Faceoff')]['Description']

# Display Data for Shots
shots = df[df['Event'].str.match('Shot')]
shots_lat = df[df['Event'].str.match('Shot')]['Lat']
shots_lon = df[df['Event'].str.match('Shot')]['Lon']
shots_text = df[df['Event'].str.match('Shot')]['Description']

# Display Data for Blocked Shots
bl_shots = df[df['Event'].str.contains('Blocked Shot')]
bl_shots_lat = df[df['Event'].str.contains('Blocked Shot')]['Lat']
bl_shots_lon = df[df['Event'].str.contains('Blocked Shot')]['Lon']
bl_shots_text = df[df['Event'].str.contains('Blocked Shot')]['Description']

# Display Data for Missed Shots
m_shots = df[df['Event'].str.contains('Missed Shot')]
m_shots_lat = df[df['Event'].str.contains('Missed Shot')]['Lat']
m_shots_lon = df[df['Event'].str.contains('Missed Shot')]['Lon']
m_shots_text = df[df['Event'].str.contains('Missed Shot')]['Description']


# Display Data for Penalties rgb(0, 255, 0)
penalties = df[df['Event'].str.contains('Penalty')]
pen_lat = df[df['Event'].str.contains('Penalty')]['Lat']
pen_lon = df[df['Event'].str.contains('Penalty')]['Lon']
pen_text = df[df['Event'].str.contains('Penalty')]['Description']

# Display Data for Giveaways rgb(51, 0, 0)
giveaway = df[df['Event'].str.contains('Giveaway')]
give_lat = df[df['Event'].str.contains('Giveaway')]['Lat']
give_lon = df[df['Event'].str.contains('Giveaway')]['Lon']
give_text = df[df['Event'].str.contains('Giveaway')]['Description']

# Display Data for Takeaways rgb(51, 0, 0)
takeaway = df[df['Event'].str.contains('Takeaway')]
take_lat = df[df['Event'].str.contains('Takeaway')]['Lat']
take_lon = df[df['Event'].str.contains('Takeaway')]['Lon']
take_text = df[df['Event'].str.contains('Takeaway')]['Description']


fig = go.Figure()


# All Event Markers
fig.add_trace(go.Scattermapbox(
        lat = events_lat,
        lon = events_lon,
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 25,
            color = '#808080',
            opacity = 0.25,
            symbol = 'circle',
        ),
    
        text = locations_name,
        hoverinfo = 'text',
        name = "All Events",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata = df,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[9]}',
    ))

# # BOS Event Markers
# fig.add_trace(go.Scattermapbox(
#         lat=BOS_lat,
#         lon=BOS_lon,
#         mode='markers',
#         marker=go.scattermapbox.Marker(
#             size=20,
#             color='rgb(255, 0, 0)',
#             opacity=0.7
#         ),
#         text=STL_text,
#         hoverinfo='text',
#         name="BOS"
#     ))

# # STL Event Markers
# fig.add_trace(go.Scattermapbox(
#         lat=STL_lat,
#         lon=STL_lon,
#         mode='markers',
#         marker=go.scattermapbox.Marker(
#             size=20,
#             color='rgb(255, 0, 0)',
#             opacity=0.7
#         ),
#         text=STL_text,
#         hoverinfo='text',
#         name="STL"
#     ))

# Home Event Markers
fig.add_trace(go.Scattermapbox(
        lat = home_lat,
        lon = home_lon,
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 20,
            color = '#0000ff',
            opacity = 0.25
        ),
        text = home_text,
        hoverinfo = 'text',
        name = "Home",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata = home,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[9]}',
    ))

# Away Event Markers
fig.add_trace(go.Scattermapbox(
        lat = away_lat,
        lon = away_lon,
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 20,
            color = 'rgb(255, 0, 0)',
            opacity = 0.25
        ),
        text = away_text,
        hoverinfo = 'text',
        name = "Away",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata = away,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[9]}',
    ))

# Goal Event Markers
fig.add_trace(go.Scattermapbox(
        lat = goal_lat,
        lon = goal_lon,
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 18,
            color = '#FFA500',
            opacity = 0.75
        ),
        text = goal_text,
        hoverinfo = 'text',
        name = "Goal",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata = goal,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[9]}',
    ))

# Hits Event Markers
fig.add_trace(go.Scattermapbox(
        lat = hits_lat,
        lon = hits_lon,
        mode = 'markers',
        marker = go.scattermapbox.Marker(
            size = 14,
            color = 'rgb(255,255,0)',
            opacity=0.7
        ),
        text=hits_text,
        hoverinfo='text',
        name="Hit",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata=hits,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[9]}',
    ))

# Faceoff Event Markers
fig.add_trace(go.Scattermapbox(
        lat=faceoff_lat,
        lon=faceoff_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=8,
            color='rgb(255,165,0)',
            opacity=0.10
        ),
        text=faceoff_text,
        hoverinfo='text',
        name="Faceoff",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata=faceoff,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[9]}',
    ))

# Shots Event Markers
fig.add_trace(go.Scattermapbox(
        lat=shots_lat,
        lon=shots_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=16,
            color='#008000',
            opacity=0.30
        ),
        text=shots_text,
        hoverinfo='text',
        name="Shot",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata=shots,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[9]}',
    ))

# Missed Shots Event Markers
fig.add_trace(go.Scattermapbox(
        lat=m_shots_lat,
        lon=m_shots_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='#008000',
            opacity=1
        ),
        text=m_shots_text,
        hoverinfo='text',
        name="Missed Shot",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata=m_shots,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[9]}',
    ))

# Blocked Shots Event Markers
fig.add_trace(go.Scattermapbox(
        lat=bl_shots_lat,
        lon=bl_shots_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='#008000',
            opacity=0.5
        ),
        text=bl_shots_text,
        hoverinfo='text',
        name="Blocked Shot",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata=bl_shots,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[9]}',
    ))

# Penalty Event Markers
fig.add_trace(go.Scattermapbox(
        lat=pen_lat,
        lon=pen_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color='rgb(0, 255, 000)',
            opacity=0.7
        ),
        text=pen_text,
        hoverinfo='text',
        name="Penalty",
#         https://plotly.com/python/hover-text-and-formatting/
        customdata=penalties,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[9]}',
    ))

# Giveaway Event Markers
fig.add_trace(go.Scattermapbox(
        lat=give_lat,
        lon=give_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=15,
            color='#FF0000',
            opacity=0.5,
            symbol = 'circle'
        ),
        text=give_text,
        hoverinfo='text',
        name="Giveaway",
    #         https://plotly.com/python/hover-text-and-formatting/
        customdata=giveaway,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[9]}',
    ))

# Takeaway Event Markers
fig.add_trace(go.Scattermapbox(
        lat=take_lat,
        lon=take_lon,
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=15,
            color='#FF0000',
            opacity=0.5,
            symbol = 'circle',
        ),
        text=take_text,
        hoverinfo='text',
        name="Takeaway",
    #         https://plotly.com/python/hover-text-and-formatting/
        customdata=takeaway,
        hovertemplate =
        '<b>Period</b>: %{customdata[4]}<br>'+
        '<b>Time</b>: %{customdata[5]}<br>'+
        '<b>Event</b>: %{customdata[0]}<br>'+
        '<b>Description</b>: %{customdata[1]}<br>'+
        '<b>Location</b>: %{customdata[9]}',
    ))

#Legend
fig.update_layout(
    width=800,
    height=500,
    margin={"r":10,"t":40,"l":10,"b":20},
    title='2019 Stanley Cup Finals (Game 2)',
    autosize=False,
    hovermode='closest',
    hoverlabel=dict(
        bgcolor="white", 
        font_size=16, 
        font_family="Rockwell"
    ),
    showlegend=True,
#     legend_title='<b>Events</b>',
    legend_orientation="v",
        legend=dict(
        x=-.25,
        y=.875,
        traceorder="normal",
        font=dict(
            family="Rockwell",
            size=18,
            color="black"
        ),
        bgcolor="white",
        bordercolor="Black",
        borderwidth=2
    ),
    mapbox=dict(
        accesstoken=mapbox_access_token,
        center=dict(
            lat=0,
            lon=0
        ),
        pitch=0,
        zoom=1,
        bearing=0,
        style='mapbox://styles/isaacv/ckar8gbe708uw1ilmmvzdflsg'
    ),
)

# Need bounds and max zoom


# Displays Home Defensive Events & Away Offensive Events on left side of ice
# Displays Away Defensive Events & Home Offensive Events on left side of ice
fig.show()

# final = fig.show()
# print(time)